In [1]:
!pip install transformers datasets accelerate evaluate -q

In [2]:
!pip install "fsspec<=2023.5.0"

In [3]:
!pip install "numpy<2.0"

In [4]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import evaluate


In [5]:
# Load IMDb dataset
dataset = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [8]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    push_to_hub=False,
)

In [10]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1)
    return accuracy.compute(predictions=predictions, references=labels)


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(5000)),  # to speed up
    eval_dataset=tokenized_datasets["test"].select(range(1000)),
    compute_metrics=compute_metrics,
)


In [12]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: alexkhundongbam260 (alexkhundongbam260-christ-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.390400,0.267268,0.912000
2,0.164800,0.344072,0.918000


TrainOutput(global_step=1250, training_loss=0.2673970226287842, metrics={'train_runtime': 1070.5699, 'train_samples_per_second': 9.341, 'train_steps_per_second': 1.168, 'total_flos': 2631110553600000.0, 'train_loss': 0.2673970226287842, 'epoch': 2.0})

In [13]:
eval_result = trainer.evaluate()
print(eval_result)

{'eval_loss': 0.3440718352794647, 'eval_accuracy': 0.918, 'eval_runtime': 30.0515, 'eval_samples_per_second': 33.276, 'eval_steps_per_second': 4.16, 'epoch': 2.0}


In [15]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    label = torch.argmax(probs).item()
    return "Positive" if label == 1 else "Negative"

# Example:
predict_sentiment("This movie was absolutely fantastic!")

'Positive'

In [16]:
predict_sentiment("The movie was good in the first half then dragged on and forgot it's focus")

'Negative'

In [17]:
from sklearn.metrics import classification_report
import numpy as np

# Run inference on test set
preds = trainer.predict(test_dataset=tokenized_datasets["test"])
pred_labels = np.argmax(preds.predictions, axis=1)

# Classification report
print(classification_report(preds.label_ids, pred_labels, target_names=['negative', 'positive']))

              precision    recall  f1-score   support

    negative       0.92      0.92      0.92     12500
    positive       0.92      0.92      0.92     12500

    accuracy                           0.92     25000
   macro avg       0.92      0.92      0.92     25000
weighted avg       0.92      0.92      0.92     25000

